# Assignment 1: The Jordan Network

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.

## Exercise 1: Initializing the network
Consider the Jordan network (with $f(x) = \tanh(x) = (e^x - e^{-x})(e^x + e^{-x})^{-1}$ and $\varphi(x) = \sigma(x) = (1+e^{-x})^{-1}$ and transposed weight matrices compared to the lecture notes)
$$
s(t) = W x(t) + R \hat y(t-1) \\
a(t) = \tanh(s(t)) \\
z(t) = V a(t) \\
\hat y(t) = \sigma(z(t))
$$
for $t \in \mathbb{N}, x(t) \in \mathbb{R}^{D}, s(t) \in \mathbb{R}^{I}, a(t) \in \mathbb{R}^{I}, z(t) \in \mathbb{R}^K, \hat y(t) \in \mathbb{R}^K$ and $W, R, V$ are matrices of appropriate sizes and $\hat y(0) = 0$. 

Write a function `init` that takes a `model` and integers $D, I, K$ as arguments and stores the matrices $W, R, V$ as members `model.W`, `model.R`, `model.V`, respectively. The matrices should be `numpy` arrays of appropriate sizes and filled with random values that are uniformly distributed between -0.01 and 0.01. 

In [3]:
%matplotlib inline
import numpy as np
from scipy.special import expit as sigmoid

def tanh_der(s):
    return np.ones_like(s)-np.square(np.tanh(s))

class Gate(object):
    pass

class Obj(object):
    pass

def init(model, D, I, K): #dunder pls
    model.W = np.random.uniform(low=-0.01,high=0.01,size=(D,I))
    model.R = np.random.uniform(low=-0.01,high=0.01,size=(I,K))
    model.V = np.random.uniform(low=-0.01,high=0.01,size=(I,K))

    t = 5 #any value of t
    test_x = np.random.uniform(size=(D))
    try:
        model.W.T @ test_x
        model.R.T*np.tanh(t)
        model.V.T*np.tanh(t)
    except Exception as e:
        print(f"Model init fail: {e}")
        
Obj.init=init

T, D, I, K = 10, 3, 5, 1

model = Obj()
model.init(D, I, K)
print("W:\n",model.W)
print("R:\n",model.R)
print("V:\n",model.V)


W:
 [[-0.00316844  0.00030774 -0.00161916 -0.00480986  0.00498588]
 [ 0.00693369  0.00548952  0.00025712 -0.00368635 -0.00263879]
 [ 0.00211893 -0.00808799  0.00036712  0.00971809  0.00211446]]
R:
 [[ 0.00060102]
 [ 0.0006513 ]
 [-0.00665671]
 [ 0.00165651]
 [ 0.00696615]]
V:
 [[-0.00154766]
 [ 0.00760051]
 [ 0.00126295]
 [ 0.00403284]
 [ 0.00757331]]


## Exercise 2: Numerical stability of the binary cross-entropy loss function

We will use the binary cross-entropy loss function to train our RNN, which is defined as 
$$
L(\hat y, y) = -y \log \hat y - (1-y) \log (1-\hat y) \quad \text{where} \quad \hat y = \sigma(z) = \frac{1}{1+e^{-z}}
$$
is the sigmoid function. Its argument $z$ is called *logit*. For reasons of numerical stability it is better to let the model emit the logit and incorporate the sigmoid function into the loss function. Explain why this is the case and how we can gain numerical stability by combining the two functions into one. *Hint:* Find a numerically stable version of the function $f(z) = \log(1+e^{z})$ and rewrite $L(z,y)$ in terms of $f(z)$. 

## ANSWER

If the model makes a large error st. z is far below zero, the sigmoid function might put out a float that is small enough to become a numerical zero in the execution of the code. This becomes problematic as the sigmoid output is passed to the loss function where the log of $\hat y$ becomes $- \infty$ and thus numerically unstable.
We thus integrate the sigmoidal function into the loss function:

$$
L(z, y) = -y \log ( \frac{1}{1+e^{-z}} ) - (1-y) \log (1-( \frac{1}{1+e^{-z}} )) \quad =
$$
$$
= -y\log(\frac{1}{1 + e^{-z}}) - (1 - y)\log(\frac{e^{-z}}{1 + e^{-z}})\\
= y  \log(1 + e^{-z}) + (1 - y)  (-\log(e^{-z}) + \log(1 + e^{-z}))\\
= y  \log(1 + e^{-z}) + (1 - y)  (z + \log(1 + e^{-z})\\
= (1 - y)  z + \log(1 + e^{-z})\\
= z - z  y + \log(1 + e^{-z})\\
$$

If z is negative and large the output of $e^{-z}$ can cause numerical overflow.
In order to prevent this we can:

$$
  z - zy + \log(1 + e^{-z})\\
= \log(e^{z}) - zy + \log(1 + e^{-z})\\
= - zy + \log(1 + e^{z})\\
$$

rewriting it in terms of $f(z)=\log (1+e^z)$:

$$
= - zy + f(z)\\
$$


## Exercise 3: The forward pass
Write a function `forward` that takes a `model`, a sequence of input vectors $(x(t))_{t=1}^T$, and a label `y` as arguments. The inputs will be represented as a `numpy` array of shape `(T, D)`. It should execute the behavior of the Jordan network and evaluate the numerically stablilized binary cross-entropy loss at the end of the sequence and return the resulting loss value. Store the sequence of hidden activations $(a(t))_{t=1}^T$ and the sequence of logits $(z(t))_{t=1}^T$ into `model.a` and `model.z`, respectively, using the same representation scheme as for the inputs. 

In [ ]:
def loss_fn(y_hat,y):

    return -(y*np.log(y_hat))-(1-y)*(np.log(1-y_hat))

def forward(self, x, y):
    self.y = y
    self.x = x
    self.a = np.zeros((T+1,I))#initial a is zeros bc there is no x for it
    self.z = np.zeros((T+1,K))#because there is one z at t=-1
    self.z[0]=self.a[0]@self.V

    self.y_hat = None
    self.s = np.empty((T,I))
    
    for t,x_t in enumerate(x):#defining t to start at one but keeping a zero-th a makes this tricky

        self.s[t] = self.W.T@x[t] + self.R@sigmoid(self.a[t]@self.V)
        #z[t] is z[t-1] in math notation bc z starts one timestampt earlier so z[t] is z[t-1], same thing for a[t]
        #in jordan network a[t-1] is replaced by y_hat[t-1]
        self.a[t+1]=np.tanh(self.s[t])
        self.z[t+1]=self.a[t]@self.V #a[-1] is now the new a, or a(t)
    self.y_hat=sigmoid(self.z)
    loss_sequence = np.apply_along_axis(loss_fn,0,self.y_hat,y=y)
    return loss_sequence[-1]

Obj.forward = forward
model.forward(np.random.uniform(-1, 1, (T, D)), 1)

## Exercise 4: The computational graph

Visualize the functional graph of the Jordan network unfolded in time. The graph should show at least 3 consecutive time steps. Use the package `networkx` in combination with `matplotlib` to draw a directed graph with labelled nodes and edges. If you need help take a look at [this guide](https://networkx.guide/visualization/basics/). Make sure to arrange the nodes in a meaningful way. 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G=nx.DiGraph()

positions={}
edge_labels={}
for _ in range(1,6):
    G.add_edge(f"yhat({_-1})",f"a({_})")
    positions[f"a({_})"]=[_+1,0]
    edge_labels[(f"yhat({_-1})",f"a({_})")]="R"
    G.add_edge(f"x({_})",f"a({_})")
    positions[f"x({_})"]=[_+1,-1]
    edge_labels[(f"x({_})",f"a({_})")]="W"
    G.add_edge(f"a({_})",f"yhat({_})")
    positions[f"yhat({_-1})"]=[_,+1]
    edge_labels[(f"a({_})",f"yhat({_})")]="V"
              
positions[f"yhat({5})"]=[_+1,+1]    

nx.draw_networkx_edge_labels(G,pos=positions,edge_labels=edge_labels,font_color='blue')

nx.draw(G,with_labels=True,pos=positions,node_size=1750,node_color="#C1C2D4",alpha =1,edge_color='k',linewidths=1)
plt.show()




## Exercise 5: Derivative of the loss

Calculate the derivative of the binary cross-entropy loss function $L(z, y)$ with respect to the logit $z$.

## ANSWER

Binary cross-entropy loss function is
$$
L(z, y) = - zy + f(z)  \quad 
$$
So the derivative with respect to the logit z is
$$
\frac{\partial }{\partial z}L(z,y) = \frac{\partial}{\partial z}\left [- zy + f(z)  \right ] =
$$
$$
\frac{\partial }{\partial z} [ \log(1 + e^{z}) ] - y =
$$
$$
\frac{e^{z}}{1+e^{z}}-y
$$
Now because $\sigma(z) = \frac{1}{1+e^-z} = \frac{e^z}{1+e^z}$

$$
=\sigma(z)-y  
$$
$$
= \hat{y}-y
$$


## Exercise 6: Gradients with respect to network parameters

Compute gradients for the weights of the Jordan network analytically. That is, derive backpropagation through time for the Jordan network. To do this, it is crucial to first find the derivative w.r.t. the network outputs $\psi^\top(t) = \partial L / \partial z(t)$ and hidden pre-activations $\delta^\top(t) = \partial L / \partial s(t)$. We use the shorthand notations $L = \sum_{t=1}^T L(t)$ and $L(t) = L(y(t), \hat y(t))$ for convenience. We use numerator-layout notation for partial derivatives (like in the lecture notes) which lets us multiply the chain rule from left to right. 

Compute the gradients $\psi^\top(t), \delta^\top(t), \nabla_W L, \nabla_R L, \nabla_V L$. *Hint:* Take a look at the  graph from the previous exercise to see the functional dependencies. 

########## YOUR SOLUTION HERE ##########
## $\psi^\top(t)$

$\psi^\top(t) = \frac {\partial L} { \partial z(t)} =\sigma(z(t))-y$

## $\delta^\top(t)$

The loss that is caused in the future by setting the hidden neurons a(t)

$$
%\delta^\top(t) = \partial L / \partial s(t) = \frac {\partial L}{\partial \hat{y}(t)} \frac{\partial \hat{y}(t)}{\partial s(t)}\\
%= \left ( \frac{\partial L(y(t),\hat{y}(t))}{\partial \hat{y}(t)} + \frac{\partial L}{\partial s(t+1)} \frac{\partial s(t+1)}{\partial \hat{y}(t)} \right) \frac{\partial \hat{y}(t)}{\partial s(t)}\\
%=\left( \frac{\partial L}{\partial z(t)}\frac{\partial z(t)}{\partial \hat{y}(t)} + \frac{\partial L}{\partial s(t+1)} \frac{\partial s(t+1)}{\partial \hat{y}(t)}  \right) \frac{\partial \hat{y}(t)}{\partial s(t)}\\
%=\left ( \psi^\top(t) V + \delta(t+1)^\top R^\top \right)diag({f}'(s(t)))
$$

$$
\delta^\top(t) = \partial L / \partial s(t) = \frac {\partial L}{\partial \hat{y}(t)} \frac{\partial \hat{y}(t)}{\partial z(t)} \frac{ \partial z(t)}{\partial a(t)} \frac {\partial a(t)}{s(t)}\\
= \left ( \psi (t) V^{\top} + \frac{\partial L}{\partial s(t+1)} \frac{\partial s(t+1)}{\partial \hat{y}(t)} \right) \frac{\partial a(t)}{\partial s(t)}\\
=\left ( \psi(t) V^{\top} + \delta(t+1)^\top R^\top \right) diag \left ({f}'(s(t)) \right )
$$

${f}'(s(t))$ being the derivative of the $\tanh$ function

$
\frac{\partial \tanh(s(t))}{\partial (s(t))} = 1 - \tanh(s(t))^2$

### For $\nabla_V L$

$\frac{\partial L}{\partial V} = \sum_{t=1}^{T}\frac{\partial L(t)}{\partial  z(t)} \frac{\partial  z(t)}{\partial V}$


Now because $z(t) = V^\top a(t)$ :

$\frac{ \partial  z(t)}{\partial V} =  
\frac {\partial(V^\top a(t))}{\partial V}
=a(t)$

Thus:

$\frac{\partial L}{\partial \upsilon_{i}} = \sum_{t=1}^{T}\frac{\partial L(t)}{\partial z(t)} a(t)
$

We already know that 

$
\frac{\partial L(t)}{\partial z(t)} = \psi^\top(t)
$

Therefore:

$\frac{\partial L}{\partial V} = \nabla_V L = \sum_{t=1}^{T} \psi(t) a(t)^{\top}$




### For R and W


$\frac{\partial L}{ \partial R} = \sum_{t=1}^{T} \frac{\partial L}{ \partial R}=\sum_{t=1}^{T} \frac{\partial L}{\partial s(t)}\frac{\partial s(t)}{ \partial R}$

now because 

$\frac{\partial L}{\partial s(t)} = \delta(t)$

and 

$\frac{\partial s(t)}{ \partial R} = \hat{y}(t-1)$

$\nabla_R L$ can be rewritten as:

$\frac{\partial L}{\partial R} = \sum_{t=1}^{T} \delta(t) \hat{y}(t-1)$

And similar to R:

$\frac{\partial L}{ \partial W} = \sum_{t=1}^{T} \delta(t) \frac{\partial s(t)}{\partial{W}}$

because 

$\frac{\partial s(t)}{\partial{W}} = x(t)$

$\nabla_W L$ can be rewritten as:

$\frac{\partial L}{ \partial W} = \sum_{t=1}^{T} \delta(t)x(t)$



## Exercise 7: The backward pass
Write a function `backward` that takes a model `self` as argument. The function should compute the gradients of the loss with respect to all model parameters and store them to `model.dW`, `model.dR`, `model.dV`, respectively. 

In [ ]:
def sig_der(z):
    return sigmoid(z)*(1-sigmoid(z))

def backward(self):
    ########## YOUR SOLUTION HERE ##########
    self.psi = np.zeros_like(self.z[1:])
    self.psi[-1]=sigmoid(self.z[-1])-self.y
    
    #self.psi = self.y_hat[1:]-self.y
    
    self.delta = np.zeros((T+1,I))#might be I,T


    for t_shift in range(1,T+1):

        self.delta[-(t_shift+1)] = (
                                        (
                                            self.psi[-t_shift]
                                            +(self.delta[-t_shift]@self.R)*sig_der(self.z[-t_shift])
                                        )
                                        * (self.V.T[0]@np.diag(tanh_der(self.s[-t_shift])))
                                   )
    self.dV = np.array([sum([np.array(self.a[t]).T*self.psi[t] for t in range(T)])])
    self.dR = np.array([sum([self.delta[t]*self.y_hat[t] for t in range(T)])])
    self.dW = sum([np.outer(self.delta[t],self.x[t]) for t in range(T)]).T
    
Obj.backward = backward
model.backward()

## Exercise 8: Gradient checking
Write a function `grad_check` that takes a model `self`, a float `eps` and another float `thresh` as arguments and computes the numerical gradients of the model parameters according to the approximation
$$
f'(x) \approx \frac{f(x + \varepsilon) - f(x - \varepsilon)}{2 \varepsilon}.
$$
If any of the analytical gradients are farther than `thresh` away from the numerical gradients the function should throw an error. 

In [ ]:
def grad_check(self, eps, thresh):
    ########## YOUR SOLUTION HERE ##########
    x,y=np.random.uniform(-1, 1, (T, D)), 1
    
    W_store=self.W
    num_grad_W=np.zeros_like(self.W)
    for col_num_grad,col_W in zip(num_grad_W,self.W):
        for row_idx in range(col_W.shape[0]):
            #negative eps
            col_W[row_idx] =col_W[row_idx]-eps
            out_low =self.forward(x,y)
            col_W[row_idx] =col_W[row_idx]+2*eps
            out_high=self.forward(x,y)
            col_num_grad[row_idx]=(out_high-out_low)/(2*eps)
            self.W=W_store
    
    self.backward()
    
    #print('dW',self.dW.T)
    #print('num grad', num_grad_W.T)   
    grad_err_W=np.abs(self.dW.T-num_grad_W.T)
    #print('grad err', grad_err_W)


    
    R_store=self.R
    num_grad_R=np.zeros_like(self.R)
    for col_num_grad,col_R in zip(num_grad_R,self.R):
        for row_idx in range(col_R.shape[0]):
            #negative eps
            col_R[row_idx] =col_R[row_idx]-eps
            out_low =self.forward(x,y)
            col_R[row_idx] =col_R[row_idx]+2*eps
            out_high=self.forward(x,y)
            col_num_grad[row_idx]=(out_high-out_low)/(2*eps)
            self.R=R_store
    
    self.backward()
    
    #print('dR',self.dR.T.shape)
    #print('num grad', num_grad_R.shape)    
    grad_err_R=np.abs(np.subtract(self.dR,num_grad_R.T[0]))
    #print('grad err', grad_err_R)

    V_store=self.V
    num_grad_V=np.zeros_like(self.V)
    for col_num_grad,col_V in zip(num_grad_V,self.V):
        for row_idx in range(col_V.shape[0]):
            #negative eps
            col_V[row_idx] =col_V[row_idx]-eps
            out_low =self.forward(x,y)
            col_V[row_idx] =col_V[row_idx]+2*eps
            out_high=self.forward(x,y)
            col_num_grad[row_idx]=(out_high-out_low)/(2*eps)
            self.V=V_store
    
    self.backward()
    
    #print('dV',self.dV.T)
    #print('num grad', num_grad_V.T)
    grad_err_V=np.abs(np.subtract(self.dV.T,num_grad_V))
    #print('grad err', grad_err_V)
    

    np.testing.assert_allclose(self.dV.T,num_grad_V,atol=thresh)
    np.testing.assert_allclose(self.dR.T,num_grad_R,atol=thresh)
    np.testing.assert_allclose(self.dW.T,num_grad_W.T,atol=thresh)
    
Obj.grad_check = grad_check

model.grad_check(1e-7, 1e-7)

## Exercise 9: Parameter update

Write a function `update` that takes a `model` and a float argument `eta`, which represents the learning rate. The method should implement the gradient descent update rule $\theta \gets \theta - \eta \nabla_{\theta}L$ for all model parameters $\theta$.

In [ ]:
def update(self, eta):
    ########## YOUR SOLUTION HERE ##########
    self.backward()
    self.W+=eta*self.dW
    self.R+=eta*self.dR.T
    self.V+=eta*self.dV.T

Obj.update = update
model.update(0.001)

## Exercise 10: Data generation

There are two classes, both occurring with probability 0.5. There is one input unit. Only the first sequence element conveys relevant information about the class. Sequence elements at positions $t > 1$ are generated by a Gaussian with mean zero and variance 0.2. The first sequence element is 1.0 (-1.0) for class 1 (2). Target at sequence end is 1.0 (0.0) for class 1 (2)

Write a function `generate_data` that takes an integer `T` as argument which represents the sequence length. Seed the `numpy` random generator with the number `0xDEADBEEF`. Implement the Python generator pattern and produce data in the way described above. The input sequences should have shape `(T, 1)` and the target values should have shape `(1,)`.

In [ ]:
np.random.seed(0xDEADBEEF)#is it supposed to spell dead beef? moooh!

def generate_data(T):
    ########## YOUR SOLUTION HERE ##########
    x_prime=np.random.choice([1,-1],1,p=[.5,.5])
    gaussian=np.random.normal(0,.2,size=(1,(T-1)))
    x = np.insert(gaussian,0,x_prime.T,axis=1)
    y = ((x_prime+1)/2).astype(int)#hacky
    yield [x.T,y]
    

## Exercise 11: Network training

Train a Jordan network with 32 hidden units. Start with input sequences of length one and tune the leraning rate and the number of update steps. Then increase the sequence length by one and tune the hyperparameters again. What is the maximal sequence length for which the Jordan network can achieve a performance that is better than random? Visualize your results. 

In [ ]:
########## YOUR SOLUTION HERE ##########

